# Week 09: Kattiana Theodore

## Setup

Run the following 2 cells to import all necessary libraries and helpers for this week's exercises

In [ ]:
!wget -q https://github.com/DM-GY-9103-2024S-R/9103-utils/raw/main/src/io_utils.py

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import wave

from IPython.display import Audio
from PIL import Image

from io_utils import wav_to_list, list_to_wav
from io_utils import get_pixels, get_Image

## Digital Audio

Air pressure waves converted to electrical pulses, which are then sampled and turned into a sequence of numbers.

<img src="./imgs/audio-00.jpg" width="720px">

### Playing an audio file

Easy !

In [ ]:
display(Audio("./data/two-bits.wav"))
display(Audio("./data/horn.wav"))

### Loading an audio file for analysis, manipulation, etc

is a bit more work

In [ ]:
sound_file_path = "./data/air-horn.wav"
with wave.open(sound_file_path, mode="rb") as wav_in:
  print(wav_in.getparams())

display(Audio(sound_file_path))

### Audio length, channels, samples, rate, depth

<img src="./imgs/audio-01.jpg" width="720px">

`Audio length`: The duration of an audio file in seconds.

`Channels`: The different signals that make up an audio file.

`Samples`: List of numbers that represent the quantized amplitude of an audio signal.

`Frame`: Collection of samples from all channels at a given time. $Number\ of\ Frames = \frac{Number\ of\ Samples}{Number\ of\ Channels}$

`Sample Rate`: How many times per second the original audio signal was recorded. $Sample\ Rate = \frac{Number\ of\ Samples}{Audio\ Length}$

`Bit Depth` / `Sample Width`: How many different unique numbers are used to represent a sample.

### Getting sample values

We first have to open a `.wav` file with `wave.open()` to get a file object.

We then use the file object `wav_in` to read the file's contents into a buffer of `bytes` with the `wav_in.readframes()` function.

We use the `frombuffer()` function to turn `bytes` into `integers`.

And, finally, we can use `list()` to put it all inside a regular Python list.

<img src="./imgs/audio-02.jpg" width="720px">

# 😫

That's a lot of cryptic lines of code just to open a file and get a list of numbers !

In [ ]:
sound_file_path = "./data/western.wav"
with wave.open(sound_file_path, mode="rb") as wav_in:
  read_buffer = wav_in.readframes(wav_in.getnframes())
  my_samples = list(np.frombuffer(read_buffer, dtype=np.int16))

print(wav_in.getparams())
print("length:", wav_in.getnframes() / wav_in.getframerate())
print(len(my_samples))
print(my_samples[:16])
print(min(my_samples), max(my_samples))

### Visualizing

At least we can visualize and play it from the list of samples.

In [ ]:
plt.plot(my_samples)
plt.show()

display(Audio(my_samples, rate=44100))

# 😫😫

For sound files with more than one channel, the `Audio()` function expects the samples in a format that is different from the one returned by `wave.open()` and `wave.readframes()`.

Argh!

We can give `Audio()` every other sample to listen to just one of the channels:
<br>`display(Audio(my_samples[::2], rate=44100))`

But, it's better to use a function to read our wave files and return a single-channel array that combines all of the channels in an audio file.

In [ ]:
sound_file_path = "./data/western.wav"
my_samples = wav_to_list(sound_file_path)

print(len(my_samples))
print(my_samples[:16])
print(min(my_samples), max(my_samples))

In [ ]:
plt.plot(my_samples)
plt.show()

display(Audio(my_samples, rate=44100))

### Manipulating

Once we have a list of samples we can process, analyze and manipulate the audio by performing list operations and simple arithmetics.

<img src="./imgs/audio-02.jpg" width="720px">

### Change volume

To change the volume of an audio file all we have to do is multiply its samples by a constant.

If the constant is greater than $1$ it will get louder, if it's between $0$ and $1$ it will get softer.

<img src="./imgs/audio-04.jpg" width="720px">

In [ ]:
sound_file_path = "./data/air-horn.wav"
my_samples = wav_to_list(sound_file_path)

plt.plot(my_samples)
plt.show()
display(Audio(sound_file_path))

changed_samples = []
for s in my_samples:
  changed_samples.append(int(s * 0.15))
changed_samples[0] = 2**15-1

plt.plot(changed_samples)
plt.show()
display(Audio(changed_samples, rate=44100))

### Change speed

If we just duplicate each sample in our sequence, while keeping the sample rate the same, we'll end up with an audio file that is twice as long as the original.

<img src="./imgs/audio-05.jpg" width="720px">

And, conversely, if we remove every other sample, we'll get an audio signal that is half of the original length.

In [ ]:
sound_file_path = "./data/horn.wav"
my_samples = wav_to_list(sound_file_path)

plt.plot(my_samples)
plt.show()
display(Audio(sound_file_path))
print(len(my_samples), "samples")

double_samples = []
for s in my_samples:
  double_samples.append(s)
  double_samples.append(s)

plt.plot(double_samples)
plt.show()
display(Audio(double_samples, rate=44100))
print(len(double_samples), "samples")

half_samples = []
for s in my_samples[::2]:
  half_samples.append(s)

plt.plot(half_samples)
plt.show()
display(Audio(half_samples, rate=44100))
print(len(half_samples), "samples")

### Reverse

Flipping the order of the samples will make the audio sound backwards.

<img src="./imgs/audio-06.jpg" width="720px">

In [ ]:
sound_file_path = "./data/two-bits.wav"
my_samples = wav_to_list(sound_file_path)

plt.plot(my_samples)
plt.show()
display(Audio(sound_file_path))
print(my_samples[:16])

rev_samples = list(reversed(my_samples))

plt.plot(rev_samples)
plt.show()
display(Audio(rev_samples, rate=44100))
print(rev_samples[-16:])


### Combining sounds

To combine two audio signals, to have them play on top of each other, we just have to add every sample $S_{A_i}$ of our first audio file with it's corresponding sample in the second audio file $S_{B_i}$.

<img src="./imgs/audio-07.jpg" width="720px">

In this situation we can use the `zip()` function, which returns a sequence that is made up of pairs of elements from other sequences.

For example, if we have:
```python
A = [10,11,12,13,14]
B = [20,21,22,23,24]
```

then, `zip(A,B)` will give us this:
```python
[(10,20), (11,21), (12,22), (13,23), (14,24)]
```

It's like a zipper, where it builds its elements from one element of each of its arguments.

In [ ]:
two_bit_file_path = "./data/two-bits.wav"
two_bit_samples = wav_to_list(two_bit_file_path)

air_horn_file_path = "./data/air-horn.wav"
air_horn_samples = wav_to_list(air_horn_file_path)

plt.plot(two_bit_samples)
plt.show()
display(Audio(two_bit_file_path))
print(len(two_bit_samples), "samples")

plt.plot(air_horn_samples)
plt.show()
display(Audio(air_horn_file_path))
print(len(air_horn_samples), "samples")

sum_samples = []
for s0, s1 in zip(two_bit_samples, air_horn_samples):
  sum_samples.append((s0 + s1) / 2)

plt.plot(sum_samples)
plt.show()
display(Audio(sum_samples, rate=44100))
print(len(sum_samples), "samples")

### Splicing

Here we want to add the second wave after the first.

In Python we can use addition to concatenate two lists:
```python
A = [0,1,2,3]
B = [4,5,6,7]
C = A + B
```

The `C` variable now holds `[0,1,2,3,4,5,6,7]`.

We can also use slicing to select parts of the two sounds before adding them.

In [ ]:
two_bit_file_path = "./data/two-bits.wav"
two_bit_samples = wav_to_list(two_bit_file_path)

air_horn_file_path = "./data/air-horn.wav"
air_horn_samples = wav_to_list(air_horn_file_path)

plt.plot(two_bit_samples)
plt.show()
display(Audio(two_bit_file_path))
print(len(two_bit_samples), "samples")

plt.plot(air_horn_samples)
plt.show()
display(Audio(air_horn_file_path))
print(len(air_horn_samples), "samples")

sum_samples = two_bit_samples + air_horn_samples

plt.plot(sum_samples)
plt.show()
display(Audio(sum_samples, rate=44100))
print(len(sum_samples), "samples")

end_idx = int(0.6 * len(two_bit_samples))
sum_samples = two_bit_samples[:end_idx] + air_horn_samples

plt.plot(sum_samples)
plt.show()
display(Audio(sum_samples, rate=44100))
print(len(sum_samples), "samples")

### Saving

We can use the `list_to_wav()` function to save a sequence of samples to a mono `.wav` file.

In [ ]:
list_to_wav(sum_samples, "out.wav")

## Images

### Loading image files

We'll use the `Image` object from the [PIL](https://pillow.readthedocs.io/en/stable/) library to open image files.

It's as simple as doing:

In [ ]:
mimg = Image.open("./data/hog.jpg")

### Image properties

<img src="./imgs/image-00.jpg" width="720px">

We can get some information about the image directly from this object.

To get its dimensions, in pixels, we can access its `size` variable, which holds $2$ values:

In [ ]:
image_width, image_height = mimg.size

print(image_width, "x", image_height)
print("total number of pixels:", image_width * image_height)

And, to get the number of channels we can call its `getbands()` function:

In [ ]:
channel_count = len(mimg.getbands())

print(channel_count, "channels")

### A note on channels

Grayscale images have $1$ channel: each pixel holds a value between $0$ and $255$ that represents how bright that pixels is.

RGB images have $3$ channels: each pixel is represented by $3$ values, one for each color red, green and blue.

RGBA images have $4$ channels: each pixel has $3$ values for its RGB components, plus an extra one for transparency.

<img src="./imgs/image-01.jpg" width="720px">

This is important because when we get the list of pixels for an image we need to know what to expect from each of the list's members.

### Visualize the image

We just have to call the built-in jupyter function `display()`

In [ ]:
display(mimg)

### Getting pixel color lists

We can also easily get a list of all the pixel color values by calling its `getdata()` member function and turning it into a `list`.

This list has $width \times height$ elements, one for each pixel on the image, and because this is an RGB image, each pixel element has $3$ values.

In [ ]:
img_pixels = list(mimg.getdata())

print(len(img_pixels))
print(img_pixels[0])

Even though we view our images as two-dimensional arrangements of colors, in memory and in files, they're just long lists of numbers.

<img src="./imgs/image-02.jpg" width="720px">

And, just like with audio files, we can create or manipulate these lists before viewing them as images.

### Creating images from pixel color lists

This is a bit trickier.

We first have to create an empty image with a given size and specific number of channels, and then pass the list of pixel values to fill it in:

In [ ]:
# This creates an empty grayscale image with size 400 x 400
rimg = Image.new("L", (400, 400))

# This fills a list with 400 * 400 random values between 0 and 255
rpix_vals = []
for i in range(400 * 400):
  rpix_vals.append(random.randint(0, 255))

# This puts the pixel values into the image object, so we can visualize it
rimg.putdata(rpix_vals)
display(rimg)

### Another example

In [ ]:
# This creates an empty RGB, 3-channel, image with size 400 x 400
rimg = Image.new("RGB", (400, 400))

# This fills a list with 400 * 400 random RGB values
rpix_vals = []
for i in range(400 * 400):
  r = random.randint(0, 255)
  g = random.randint(0, 255)
  b = random.randint(0, 255)
  rpix_vals.append((r, g, b))

# This puts the pixel values into the image object, so we can visualize it
rimg.putdata(rpix_vals)
display(rimg)

## 😵‍💫😖

And, just like with audio files and sample lists, it's kind of annoying to always be turning pixels into images and images into pixels like this.

Additionally, if the content of the pixel list passed to the function doesn't match the expected number of pixels or channels, the conversion will fail.

Luckily, we can use some helper functions to make this easier.

### To get pixel lists:
`get_pixels(input)` : returns a list of pixel color values when given an `Image` object or the path to an image file.

### To get `Image` objects:
`get_Image(filepath)` : returns an `Image` object from the given file path.

`get_Image(pixels, width, height)` : returns an `Image` object with size `width` $\times$ `height` created from the values in the `pixels` list.

We always have to specify at least the `width` value when creating an image from a pixel array, or else the function won't be able to know if we want an image that's $600 \times 400$ or $400 \times 600$.

In [ ]:
mimg = get_Image("./data/hog.jpg")
mpxs = get_pixels("./data/hog.jpg")
# or
# mpxs = get_pixels(mimg)

print("image size:", mimg.size, "pixel count:", len(mpxs))
print("first pixels:", mpxs[0])

display(mimg)

### Filtering by pixel

We can create new images by changing the values of the pixels in our list.

For example, if we want to separate the red component of our image, we can go through all of the pixel values and lower their green and blue components.

In [ ]:
redpxs = []

for r,g,b in mpxs:
  redpxs.append((r, 0, 0))

redimg = get_Image(redpxs, mimg.size[0], mimg.size[1])

display(redimg)

We can exaggerate colors, by saturating a chosen channel in every pixel.

In [ ]:
satpxs = []

for r,g,b in mpxs:
  if max(r,g,b) == g:
    newg = int(min(255, 1.75 * g))
    satpxs.append((r, newg, b))
  else:
    satpxs.append((r, g, b))

satimg = get_Image(satpxs, mimg.size[0], mimg.size[1])

display(satimg)

We can also remove the colors by making all $3$ channels be equal to their average value.

In [ ]:
gpxs = []

for r,g,b in mpxs:
  gval = (r + g + b) // 3
  gpxs.append((gval, gval, gval))

gimg = get_Image(gpxs, mimg.size[0], mimg.size[1])

display(gimg)

### Saving

To save an image file all we have to do is call the `.save()` function of an `Image` object.

In [ ]:
gimg.save("gray-hog.jpg")